In [10]:
import torch

from helpers.model.util import select_device
from helpers.experiment import Experiment
from helpers.data.dset.config import Config_Dataset
from helpers.data.dset.constants import (
    Names_Datasets,
    Names_Levels,
    Names_q_Squared_Vetos,
    Names_Splits
)
from helpers.model.config import Config_Model
from helpers.model.constants import Names_Models

In [11]:
path_dir_plots = "../../state/new_physics/plots"
path_dir_dsets_main = "../../state/new_physics/data/processed"
path_dir_raw_signal = "../../state/new_physics/data/raw/signal"
path_dir_models_main = "../../state/new_physics/models"

device = select_device()

experiment = Experiment(
    path_dir_plots=path_dir_plots,
    device=device,
)

Device:  cuda


In [12]:
config_dset_images_train = Config_Dataset(
    name=Names_Datasets().images,
    level=Names_Levels().generator,
    q_squared_veto=Names_q_Squared_Vetos().loose,
    balanced_classes=True,
    std_scale=True,
    split=Names_Splits().train,
    path_dir_dsets_main=path_dir_dsets_main,
    path_dir_raw_signal=path_dir_raw_signal,
    shuffle=True,
    num_events_per_set=100,
    num_sets_per_label=3,
    num_bins_image=10,
)

config_dset_images_eval = Config_Dataset(
    name=Names_Datasets().images,
    level=Names_Levels().generator,
    q_squared_veto=Names_q_Squared_Vetos().loose,
    balanced_classes=True,
    std_scale=True,
    split=Names_Splits().eval_,
    path_dir_dsets_main=path_dir_dsets_main,
    path_dir_raw_signal=path_dir_raw_signal,
    shuffle=True,
    num_events_per_set=100,
    num_sets_per_label=3,
    num_bins_image=10,
)

config_model_images = Config_Model(
    name=Names_Models().cnn,
    path_dir_models_main=path_dir_models_main,
    config_dset_train=config_dset_images_train,
    loss_fn=torch.nn.MSELoss(),
    learn_rate=4e-4,
    size_batch_train=10,
    size_batch_eval=10,
    num_epochs=10,
    num_epochs_checkpoint=5,
)

In [8]:
experiment.train(
    config_model=config_model_images,
    config_dset_eval=config_dset_images_eval,
    generate_dsets=False,
)

Loaded tensor of shape: torch.Size([132, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\100_train_features.pt
Loaded tensor of shape: torch.Size([132]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\100_train_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([132, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\100_eval_features.pt
Loaded tensor of shape: torch.Size([132]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\100_eval_labels.pt
Loaded dataset: images

Epoch 0 complete:
    Train loss: 1.1235971834944072
    Eval loss: 0.9191025400919549

Peak GPU memory usage:
0.06448 GB
Saved loss table.
Saved checkpoint at epoch: 0

Epoch 1 complete:
    Train loss: 1.0208124470160647
    Eval loss: 0.9564138131691635

Peak GPU memory usage:
0.06448 GB

Epoch 2 complete:
    Train loss: 0.8403397825789225
    Eval loss: 0.8085938111850002

Peak GPU memory usage:
0.064

In [9]:
import pandas

df_charge = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_calc.pkl")

df_mix = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_calc.pkl")


FileNotFoundError: [Errno 2] No such file or directory: '../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_calc.pkl'

In [ ]:
def split(df):
    ind_half = int(len(df)/2)
    df_train = df[:ind_half].copy()
    df_eval = df[ind_half:].copy()
    return df_train, df_eval

df_charge_train, df_charge_eval = split(df_charge.loc["det"])

df_mix_train, df_mix_eval = split(df_mix.loc["det"])


In [ ]:
df_charge.loc["det"]

,__experiment__,__run__,__event__,__production__,__candidate__,__ncandidates__,__weight__,Mbc,deltaE,M,...,costheta_mu,costheta_mu_mc,costheta_K,costheta_K_mc,coschi,coschi_mc,chi,chi_mc,invM_K_pi_shifted,invM_K_pi_shifted_mc
0,1003,0,801941180,40586,0,1,1.0,5.213125,-0.008177,5.204828,...,0.018595,0.440611,-0.336317,-0.103388,-0.130366,0.914369,4.581651,5.866336,-0.079235,-0.078837
1,1003,0,801973227,40586,0,2,1.0,5.217181,-0.003621,5.213509,...,0.171435,0.453686,0.014434,-0.073350,0.959607,0.106720,5.997990,4.819313,-0.094279,-0.095087
2,1003,0,801973227,40586,1,2,1.0,5.227460,-0.027798,5.199328,...,0.318852,0.453686,0.386636,-0.073350,-0.126841,0.106720,1.697979,4.819313,0.052877,0.052307
3,1003,0,808409544,40586,0,1,1.0,5.244023,-0.031181,5.212568,...,-0.308693,-0.385081,0.324510,0.340747,-0.859764,-0.503960,3.677581,4.184212,-0.087379,-0.087142
4,1003,0,19304934,40586,0,1,1.0,5.218371,0.019338,5.237972,...,0.772394,0.388443,0.696041,0.485756,-0.777075,-0.810546,2.460801,2.515880,-0.004088,0.001022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,1003,0,35843581,32285,0,1,1.0,5.203995,0.047552,5.252324,...,0.075642,0.467534,0.105242,0.505098,0.871491,-0.595842,0.512563,4.074076,-0.055787,0.057609
0,1003,0,32723078,32285,0,1,1.0,5.236001,-0.014693,5.221157,...,0.988921,-0.042557,-0.355374,-0.325941,-0.515377,0.965381,2.112244,6.019288,0.058665,0.058473
1,1003,0,26150398,32285,0,1,1.0,5.214756,-0.019939,5.194528,...,0.099703,0.046119,0.578006,-0.614274,-0.405156,0.362481,4.295239,1.199867,0.018730,0.338043
0,1003,0,28892454,32285,0,1,1.0,5.202400,-0.014981,5.187166,...,0.230526,0.080850,-0.210434,0.242979,-0.742664,0.292931,3.875349,5.009679,-0.000727,-0.003733


In [ ]:
df_charge_train

,__experiment__,__run__,__event__,__production__,__candidate__,__ncandidates__,__weight__,Mbc,deltaE,M,...,costheta_mu,costheta_mu_mc,costheta_K,costheta_K_mc,coschi,coschi_mc,chi,chi_mc,invM_K_pi_shifted,invM_K_pi_shifted_mc
0,1003,0,801941180,40586,0,1,1.0,5.213125,-0.008177,5.204828,...,0.018595,0.440611,-0.336317,-0.103388,-0.130366,0.914369,4.581651,5.866336,-0.079235,-0.078837
1,1003,0,801973227,40586,0,2,1.0,5.217181,-0.003621,5.213509,...,0.171435,0.453686,0.014434,-0.073350,0.959607,0.106720,5.997990,4.819313,-0.094279,-0.095087
2,1003,0,801973227,40586,1,2,1.0,5.227460,-0.027798,5.199328,...,0.318852,0.453686,0.386636,-0.073350,-0.126841,0.106720,1.697979,4.819313,0.052877,0.052307
3,1003,0,808409544,40586,0,1,1.0,5.244023,-0.031181,5.212568,...,-0.308693,-0.385081,0.324510,0.340747,-0.859764,-0.503960,3.677581,4.184212,-0.087379,-0.087142
4,1003,0,19304934,40586,0,1,1.0,5.218371,0.019338,5.237972,...,0.772394,0.388443,0.696041,0.485756,-0.777075,-0.810546,2.460801,2.515880,-0.004088,0.001022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,1003,0,716807533,40556,0,1,1.0,5.226812,0.016677,5.243689,...,0.522827,0.722809,-0.333511,0.932647,-0.996532,-0.924975,3.058284,3.531418,0.012963,0.016087
15,1003,0,716991984,40556,0,1,1.0,5.218240,-0.031259,5.186549,...,0.426094,0.351112,0.050971,0.214639,0.566264,-0.332197,0.968830,1.909428,-0.090900,-0.076320
16,1003,0,114076568,40556,0,1,1.0,5.258451,-0.007075,5.251333,...,0.807953,0.692627,0.199533,-0.247337,-0.269930,0.004027,1.844117,4.716416,0.073718,0.303692
17,1003,0,114422724,40556,0,1,1.0,5.254491,-0.030454,5.223832,...,0.936232,0.851491,-0.235139,-0.803007,-0.668719,0.998362,3.979904,6.225935,-0.053785,-0.051063


In [ ]:
df_charge_eval

,__experiment__,__run__,__event__,__production__,__candidate__,__ncandidates__,__weight__,Mbc,deltaE,M,...,costheta_mu,costheta_mu_mc,costheta_K,costheta_K_mc,coschi,coschi_mc,chi,chi_mc,invM_K_pi_shifted,invM_K_pi_shifted_mc
19,1003,0,135125368,40556,0,1,1.0,5.216355,-0.017540,5.198567,...,0.802215,0.370186,0.020150,-0.210025,-0.788703,-0.791146,3.803692,3.799709,0.066220,0.069052
20,1003,0,711431006,40556,0,1,1.0,5.215776,0.028272,5.244447,...,-0.226152,-0.263764,-0.593473,0.304751,-0.996146,-0.870754,3.053767,3.655655,0.025560,0.025884
21,1003,0,711493879,40556,0,1,1.0,5.201923,-0.048107,5.152995,...,-0.162380,0.479734,-0.782716,-0.567720,-0.278963,0.744795,1.853511,5.552616,-0.013738,-0.034396
22,1003,0,494694214,40556,0,1,1.0,5.206313,0.016254,5.222827,...,0.721551,0.881120,0.187550,0.009748,-0.036367,0.773693,1.607171,5.597038,0.092325,0.081899
23,1003,0,484374433,40556,0,1,1.0,5.222490,-0.042054,5.179890,...,0.245837,0.586748,-0.833715,-0.629257,-0.283585,0.776034,4.424858,0.682444,0.094175,0.256945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,1003,0,35843581,32285,0,1,1.0,5.203995,0.047552,5.252324,...,0.075642,0.467534,0.105242,0.505098,0.871491,-0.595842,0.512563,4.074076,-0.055787,0.057609
0,1003,0,32723078,32285,0,1,1.0,5.236001,-0.014693,5.221157,...,0.988921,-0.042557,-0.355374,-0.325941,-0.515377,0.965381,2.112244,6.019288,0.058665,0.058473
1,1003,0,26150398,32285,0,1,1.0,5.214756,-0.019939,5.194528,...,0.099703,0.046119,0.578006,-0.614274,-0.405156,0.362481,4.295239,1.199867,0.018730,0.338043
0,1003,0,28892454,32285,0,1,1.0,5.202400,-0.014981,5.187166,...,0.230526,0.080850,-0.210434,0.242979,-0.742664,0.292931,3.875349,5.009679,-0.000727,-0.003733


In [ ]:
df_mix.loc["det"]

,__experiment__,__run__,__event__,__production__,__candidate__,__ncandidates__,__weight__,Mbc,deltaE,M,...,costheta_mu,costheta_mu_mc,costheta_K,costheta_K_mc,coschi,coschi_mc,chi,chi_mc,invM_K_pi_shifted,invM_K_pi_shifted_mc
0,1003,0,51527862,24824,0,1,1.0,5.223935,0.000886,5.224832,...,0.374999,0.517455,0.696839,-0.140953,0.010912,-0.546490,1.559884,2.148964,-0.093001,0.377231
0,1003,0,53353985,24824,0,1,1.0,5.214425,0.024997,5.239781,...,-0.314766,-0.072854,-0.502512,-0.304523,0.942747,0.903540,5.943162,5.840349,0.094037,-0.253359
0,1003,0,13361946,24823,0,1,1.0,5.230380,0.048252,5.279174,...,-0.046771,-0.339135,-0.386589,0.562878,-0.872680,-0.155653,3.651725,1.727085,0.012069,0.010854
0,1003,0,19336317,24824,0,2,1.0,5.203705,0.023916,5.228015,...,-0.099566,-0.021283,-0.520812,-0.716078,-0.736489,0.184025,2.398661,1.385716,0.072317,0.280779
1,1003,0,19336317,24824,1,2,1.0,5.203523,0.017299,5.221109,...,0.099896,0.021283,-0.052892,-0.857428,-0.881993,0.527363,2.650871,5.267883,0.057340,0.280779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,1003,0,214432328,36497,0,1,1.0,5.200495,0.003758,5.204317,...,0.426390,0.203453,0.879976,0.487913,-0.018882,0.881197,1.589680,0.492408,-0.000775,-0.004003
24,1003,0,214571883,36497,0,1,1.0,5.218423,0.009134,5.227682,...,-0.590810,-0.481819,-0.674666,0.251877,-0.841963,-0.915405,3.711477,3.555877,-0.011762,0.184996
25,1003,0,280840548,36497,0,1,1.0,5.215068,-0.025643,5.189056,...,0.758041,0.837996,-0.464030,0.441254,-0.770359,0.677937,3.832985,5.457342,0.025649,0.018921
26,1003,0,342250690,36497,0,1,1.0,5.214369,0.001298,5.215686,...,-0.447015,-0.607576,-0.641822,-0.781186,-0.161208,-0.761716,4.550474,3.846431,0.071971,0.073247


In [ ]:
df_mix_train

,__experiment__,__run__,__event__,__production__,__candidate__,__ncandidates__,__weight__,Mbc,deltaE,M,...,costheta_mu,costheta_mu_mc,costheta_K,costheta_K_mc,coschi,coschi_mc,chi,chi_mc,invM_K_pi_shifted,invM_K_pi_shifted_mc
0,1003,0,51527862,24824,0,1,1.0,5.223935,0.000886,5.224832,...,0.374999,0.517455,0.696839,-0.140953,0.010912,-0.546490,1.559884,2.148964,-0.093001,0.377231
0,1003,0,53353985,24824,0,1,1.0,5.214425,0.024997,5.239781,...,-0.314766,-0.072854,-0.502512,-0.304523,0.942747,0.903540,5.943162,5.840349,0.094037,-0.253359
0,1003,0,13361946,24823,0,1,1.0,5.230380,0.048252,5.279174,...,-0.046771,-0.339135,-0.386589,0.562878,-0.872680,-0.155653,3.651725,1.727085,0.012069,0.010854
0,1003,0,19336317,24824,0,2,1.0,5.203705,0.023916,5.228015,...,-0.099566,-0.021283,-0.520812,-0.716078,-0.736489,0.184025,2.398661,1.385716,0.072317,0.280779
1,1003,0,19336317,24824,1,2,1.0,5.203523,0.017299,5.221109,...,0.099896,0.021283,-0.052892,-0.857428,-0.881993,0.527363,2.650871,5.267883,0.057340,0.280779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,1003,0,250671033,40606,0,1,1.0,5.208002,0.047610,5.256353,...,0.417525,0.341045,0.263465,0.874970,-0.468923,-0.923523,2.058867,2.747963,-0.000658,-0.000624
11,1003,0,263263636,40606,0,1,1.0,5.220248,0.044189,5.265020,...,-0.510932,-0.714826,-0.890093,0.800061,0.294654,0.757173,1.271703,5.571363,0.086222,0.248889
12,1003,0,263265278,40606,0,1,1.0,5.206976,0.010447,5.217588,...,-0.758103,-0.496666,0.630879,-0.454582,0.158988,-0.425625,4.872055,4.272736,0.087518,0.034563
13,1003,0,263448211,40606,0,1,1.0,5.241786,0.044393,5.286582,...,-0.059899,0.304752,-0.171402,-0.575719,0.927623,-0.409303,5.900386,4.290699,0.058528,0.289602


In [ ]:
df_mix_eval

,__experiment__,__run__,__event__,__production__,__candidate__,__ncandidates__,__weight__,Mbc,deltaE,M,...,costheta_mu,costheta_mu_mc,costheta_K,costheta_K_mc,coschi,coschi_mc,chi,chi_mc,invM_K_pi_shifted,invM_K_pi_shifted_mc
15,1003,0,414128099,40606,0,1,1.0,5.242879,0.012483,5.255474,...,0.882504,0.946003,-0.847876,0.509017,-0.439446,-0.602365,2.025779,4.065929,-0.006610,-0.005030
16,1003,0,414628855,40606,0,1,1.0,5.219624,-0.049558,5.169393,...,-0.280441,-0.292702,-0.639606,-0.292751,0.421383,0.188839,1.135826,1.380816,0.015810,0.201691
17,1003,0,455595503,40606,0,1,1.0,5.209586,-0.005685,5.203814,...,-0.860750,NaN,0.988364,NaN,0.842997,NaN,0.567965,NaN,-0.006594,NaN
18,1003,0,513446185,40606,0,1,1.0,5.220782,0.018142,5.239163,...,0.882611,0.697400,-0.244565,0.872852,-0.475908,-0.227466,2.066792,1.800271,0.051071,0.052062
19,1003,0,514213842,40606,0,1,1.0,5.239551,0.007627,5.247252,...,0.224670,0.154521,-0.865779,0.318047,0.955947,-0.015979,0.297926,4.696410,0.034077,0.220437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,1003,0,214432328,36497,0,1,1.0,5.200495,0.003758,5.204317,...,0.426390,0.203453,0.879976,0.487913,-0.018882,0.881197,1.589680,0.492408,-0.000775,-0.004003
24,1003,0,214571883,36497,0,1,1.0,5.218423,0.009134,5.227682,...,-0.590810,-0.481819,-0.674666,0.251877,-0.841963,-0.915405,3.711477,3.555877,-0.011762,0.184996
25,1003,0,280840548,36497,0,1,1.0,5.215068,-0.025643,5.189056,...,0.758041,0.837996,-0.464030,0.441254,-0.770359,0.677937,3.832985,5.457342,0.025649,0.018921
26,1003,0,342250690,36497,0,1,1.0,5.214369,0.001298,5.215686,...,-0.447015,-0.607576,-0.641822,-0.781186,-0.161208,-0.761716,4.550474,3.846431,0.071971,0.073247


In [ ]:
pandas.to_pickle(df_charge_train, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_train.pkl")

In [ ]:
pandas.to_pickle(df_charge_eval, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_eval.pkl")


In [ ]:
pandas.to_pickle(df_mix_train, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_train.pkl")


In [ ]:
pandas.to_pickle(df_mix_eval, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_eval.pkl")


In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_train.pkl")

,__experiment__,__run__,__event__,__production__,__candidate__,__ncandidates__,__weight__,Mbc,deltaE,M,...,costheta_mu,costheta_mu_mc,costheta_K,costheta_K_mc,coschi,coschi_mc,chi,chi_mc,invM_K_pi_shifted,invM_K_pi_shifted_mc
0,1003,0,51527862,24824,0,1,1.0,5.223935,0.000886,5.224832,...,0.374999,0.517455,0.696839,-0.140953,0.010912,-0.546490,1.559884,2.148964,-0.093001,0.377231
0,1003,0,53353985,24824,0,1,1.0,5.214425,0.024997,5.239781,...,-0.314766,-0.072854,-0.502512,-0.304523,0.942747,0.903540,5.943162,5.840349,0.094037,-0.253359
0,1003,0,13361946,24823,0,1,1.0,5.230380,0.048252,5.279174,...,-0.046771,-0.339135,-0.386589,0.562878,-0.872680,-0.155653,3.651725,1.727085,0.012069,0.010854
0,1003,0,19336317,24824,0,2,1.0,5.203705,0.023916,5.228015,...,-0.099566,-0.021283,-0.520812,-0.716078,-0.736489,0.184025,2.398661,1.385716,0.072317,0.280779
1,1003,0,19336317,24824,1,2,1.0,5.203523,0.017299,5.221109,...,0.099896,0.021283,-0.052892,-0.857428,-0.881993,0.527363,2.650871,5.267883,0.057340,0.280779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,1003,0,250671033,40606,0,1,1.0,5.208002,0.047610,5.256353,...,0.417525,0.341045,0.263465,0.874970,-0.468923,-0.923523,2.058867,2.747963,-0.000658,-0.000624
11,1003,0,263263636,40606,0,1,1.0,5.220248,0.044189,5.265020,...,-0.510932,-0.714826,-0.890093,0.800061,0.294654,0.757173,1.271703,5.571363,0.086222,0.248889
12,1003,0,263265278,40606,0,1,1.0,5.206976,0.010447,5.217588,...,-0.758103,-0.496666,0.630879,-0.454582,0.158988,-0.425625,4.872055,4.272736,0.087518,0.034563
13,1003,0,263448211,40606,0,1,1.0,5.241786,0.044393,5.286582,...,-0.059899,0.304752,-0.171402,-0.575719,0.927623,-0.409303,5.900386,4.290699,0.058528,0.289602


In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_eval.pkl")


,__experiment__,__run__,__event__,__production__,__candidate__,__ncandidates__,__weight__,Mbc,deltaE,M,...,costheta_mu,costheta_mu_mc,costheta_K,costheta_K_mc,coschi,coschi_mc,chi,chi_mc,invM_K_pi_shifted,invM_K_pi_shifted_mc
15,1003,0,414128099,40606,0,1,1.0,5.242879,0.012483,5.255474,...,0.882504,0.946003,-0.847876,0.509017,-0.439446,-0.602365,2.025779,4.065929,-0.006610,-0.005030
16,1003,0,414628855,40606,0,1,1.0,5.219624,-0.049558,5.169393,...,-0.280441,-0.292702,-0.639606,-0.292751,0.421383,0.188839,1.135826,1.380816,0.015810,0.201691
17,1003,0,455595503,40606,0,1,1.0,5.209586,-0.005685,5.203814,...,-0.860750,NaN,0.988364,NaN,0.842997,NaN,0.567965,NaN,-0.006594,NaN
18,1003,0,513446185,40606,0,1,1.0,5.220782,0.018142,5.239163,...,0.882611,0.697400,-0.244565,0.872852,-0.475908,-0.227466,2.066792,1.800271,0.051071,0.052062
19,1003,0,514213842,40606,0,1,1.0,5.239551,0.007627,5.247252,...,0.224670,0.154521,-0.865779,0.318047,0.955947,-0.015979,0.297926,4.696410,0.034077,0.220437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,1003,0,214432328,36497,0,1,1.0,5.200495,0.003758,5.204317,...,0.426390,0.203453,0.879976,0.487913,-0.018882,0.881197,1.589680,0.492408,-0.000775,-0.004003
24,1003,0,214571883,36497,0,1,1.0,5.218423,0.009134,5.227682,...,-0.590810,-0.481819,-0.674666,0.251877,-0.841963,-0.915405,3.711477,3.555877,-0.011762,0.184996
25,1003,0,280840548,36497,0,1,1.0,5.215068,-0.025643,5.189056,...,0.758041,0.837996,-0.464030,0.441254,-0.770359,0.677937,3.832985,5.457342,0.025649,0.018921
26,1003,0,342250690,36497,0,1,1.0,5.214369,0.001298,5.215686,...,-0.447015,-0.607576,-0.641822,-0.781186,-0.161208,-0.761716,4.550474,3.846431,0.071971,0.073247


In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_train.pkl")

,__experiment__,__run__,__event__,__production__,__candidate__,__ncandidates__,__weight__,Mbc,deltaE,M,...,costheta_mu,costheta_mu_mc,costheta_K,costheta_K_mc,coschi,coschi_mc,chi,chi_mc,invM_K_pi_shifted,invM_K_pi_shifted_mc
0,1003,0,801941180,40586,0,1,1.0,5.213125,-0.008177,5.204828,...,0.018595,0.440611,-0.336317,-0.103388,-0.130366,0.914369,4.581651,5.866336,-0.079235,-0.078837
1,1003,0,801973227,40586,0,2,1.0,5.217181,-0.003621,5.213509,...,0.171435,0.453686,0.014434,-0.073350,0.959607,0.106720,5.997990,4.819313,-0.094279,-0.095087
2,1003,0,801973227,40586,1,2,1.0,5.227460,-0.027798,5.199328,...,0.318852,0.453686,0.386636,-0.073350,-0.126841,0.106720,1.697979,4.819313,0.052877,0.052307
3,1003,0,808409544,40586,0,1,1.0,5.244023,-0.031181,5.212568,...,-0.308693,-0.385081,0.324510,0.340747,-0.859764,-0.503960,3.677581,4.184212,-0.087379,-0.087142
4,1003,0,19304934,40586,0,1,1.0,5.218371,0.019338,5.237972,...,0.772394,0.388443,0.696041,0.485756,-0.777075,-0.810546,2.460801,2.515880,-0.004088,0.001022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,1003,0,716807533,40556,0,1,1.0,5.226812,0.016677,5.243689,...,0.522827,0.722809,-0.333511,0.932647,-0.996532,-0.924975,3.058284,3.531418,0.012963,0.016087
15,1003,0,716991984,40556,0,1,1.0,5.218240,-0.031259,5.186549,...,0.426094,0.351112,0.050971,0.214639,0.566264,-0.332197,0.968830,1.909428,-0.090900,-0.076320
16,1003,0,114076568,40556,0,1,1.0,5.258451,-0.007075,5.251333,...,0.807953,0.692627,0.199533,-0.247337,-0.269930,0.004027,1.844117,4.716416,0.073718,0.303692
17,1003,0,114422724,40556,0,1,1.0,5.254491,-0.030454,5.223832,...,0.936232,0.851491,-0.235139,-0.803007,-0.668719,0.998362,3.979904,6.225935,-0.053785,-0.051063


In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_eval.pkl")

,__experiment__,__run__,__event__,__production__,__candidate__,__ncandidates__,__weight__,Mbc,deltaE,M,...,costheta_mu,costheta_mu_mc,costheta_K,costheta_K_mc,coschi,coschi_mc,chi,chi_mc,invM_K_pi_shifted,invM_K_pi_shifted_mc
19,1003,0,135125368,40556,0,1,1.0,5.216355,-0.017540,5.198567,...,0.802215,0.370186,0.020150,-0.210025,-0.788703,-0.791146,3.803692,3.799709,0.066220,0.069052
20,1003,0,711431006,40556,0,1,1.0,5.215776,0.028272,5.244447,...,-0.226152,-0.263764,-0.593473,0.304751,-0.996146,-0.870754,3.053767,3.655655,0.025560,0.025884
21,1003,0,711493879,40556,0,1,1.0,5.201923,-0.048107,5.152995,...,-0.162380,0.479734,-0.782716,-0.567720,-0.278963,0.744795,1.853511,5.552616,-0.013738,-0.034396
22,1003,0,494694214,40556,0,1,1.0,5.206313,0.016254,5.222827,...,0.721551,0.881120,0.187550,0.009748,-0.036367,0.773693,1.607171,5.597038,0.092325,0.081899
23,1003,0,484374433,40556,0,1,1.0,5.222490,-0.042054,5.179890,...,0.245837,0.586748,-0.833715,-0.629257,-0.283585,0.776034,4.424858,0.682444,0.094175,0.256945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,1003,0,35843581,32285,0,1,1.0,5.203995,0.047552,5.252324,...,0.075642,0.467534,0.105242,0.505098,0.871491,-0.595842,0.512563,4.074076,-0.055787,0.057609
0,1003,0,32723078,32285,0,1,1.0,5.236001,-0.014693,5.221157,...,0.988921,-0.042557,-0.355374,-0.325941,-0.515377,0.965381,2.112244,6.019288,0.058665,0.058473
1,1003,0,26150398,32285,0,1,1.0,5.214756,-0.019939,5.194528,...,0.099703,0.046119,0.578006,-0.614274,-0.405156,0.362481,4.295239,1.199867,0.018730,0.338043
0,1003,0,28892454,32285,0,1,1.0,5.202400,-0.014981,5.187166,...,0.230526,0.080850,-0.210434,0.242979,-0.742664,0.292931,3.875349,5.009679,-0.000727,-0.003733
